# Building LangChain Agents to Automate Tasks in Python

## Introduction

LangChain's 90k GitHub stars is all the credibility it needs - right now, it is the hottest framework to build LLM-based applications. Its comprehensive set of tools and components allows you to build end-to-end AI solutions using almost any LLM. 

Perhaps, at the heart of LangChain's capabilities are LangChain Agents. These agents are autonomous or semi-autonomous tools that can perform tasks, make decisions, and interact with other tools and APIs. They represent a significant leap forward in automating complex workflows with LLMs. 

In this article, you will learn how to build your own LangChain agents that can perform tasks not strictly possible with todays chat applications like ChatGPT. 

## What are LangChain Agents?

Before we write code, let's understand the __agent framework__ and why choose it over traditional programming paradigms. 

### Chains vs. Agents


The defining trait of agents is their ability to choose the best order of actions to solve a problem given a set of tools.

For example, let's say we have the following:

- A weather API
- ML model for clothing recommendations
- Strava API for biking routes
- User preferences database
- Image recognition model
- Language model (text generation)

Traditional problem-solving would involve using a chain of select tools from the list:

Chain 1: Weather-based clothing recommender
1. Call weather API
2. Input weather data into ML Clothing Model
3. Generate clothing recommendations
4. Present results to user

Chain 2: Weather-based biking route suggester
1. Call weather API
2. Call Strava API for popular routes
3. Filter routes based on weather conditions
4. Present suitable routes to user

Chain 3: Outfit Photo Analyzer
1. Receive user's outfit photo
2. Use Image Recognition Model to identify clothing items
3. Compare with User Preference Database
4. Generate feedback using Text Generation Model
5. Present analysis to user

Each chain solves a specific problem, using a predetermined sequence of steps and a subset of the available tools. They cannot adapt beyond their defined scope. They also require three separate branches of development, which is inefficient in terms of time and resources.

Now, imagine an agentic system (agent) with access to all these tools. It would be able to:

1. Understand user's query or problem
2. Assess which tools are relevant to the problem
3. Dynamically create a workflow using the most appropriate tools
4. Execute the workflow, making real-time adjustments if needed
5. Evaluate the outcome and learn for future interactions

For example, if a user asks "What should I wear for my bike ride today?", the agent might check the weather API, analyze suitable biking routes through Strava, recommend appropriate clothing considering user's past preferences and generate a personalized response. 

The agent can:
- Handle a wide variety of problems using the same set of tools
- Create custom workflows for each unique situation
- Adapt its approach based on the specific context and user needs
- Learn from interactions to improve future performance

LangChain's capacity to transform language models—which, by themselves, only produce text—into reasoning engines that can use the resources at their disposal to take appropriate action is one of its main applications. Put differently, LangChain enables the development of strong autonomous agents.

### Key components

#### __Chat models__

There are a lot of moving parts involved in creating a LangChain agent. The first and most obvious is a language model. 

In [5]:
from langchain_openai import OpenAI

llm = OpenAI(api_key=api_key, model="gpt-3.5-turbo-instruct")

question = "What is special about the number 73?"
output = llm.invoke(question)

print(output[:100])


1. Prime number: 73 is a prime number, which means it is only divisible by 1 and itself. This makes


Language models, like OpenAI's GPT-3.5 Turbo, take and generate strings. They are typically older and work best to answer individual user queries. 

For this reason, agents usually use chat models, which can take a sequence of messages as inputs and return chat messages as outputs (as opposed to using plain text):

In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessageChunk, SystemMessage

# Initialize the model
chat_model = ChatOpenAI(api_key=api_key, model='gpt-4o-mini')

# Write the messages
messages = [SystemMessage(content='You are a grumpy pirate.'),
            HumanMessage(content="What's up?")]

output = chat_model.invoke(messages)

Put differently, they allow us to have conversations with chat models. Above, we are initializing GPT-4o-mini with a system message followed by a user query. Note the use of `SystemMessage` and `HumanMessage` classes. 

Chat models return messages as outputs:

In [11]:
type(output)

langchain_core.messages.ai.AIMessage

In [14]:
print(output.content)

Arrr, nothin' much but the same ol' waves and the smell of salt in the air! Just tryin' to avoid scallywags and keep me treasure safe. What be ye wantin', eh? Spit it out!


Besides, they return other useful metadata accessible with dot-notation:

In [17]:
output.dict()

{'content': "Arrr, nothin' much but the same ol' waves and the smell of salt in the air! Just tryin' to avoid scallywags and keep me treasure safe. What be ye wantin', eh? Spit it out!",
 'additional_kwargs': {},
 'response_metadata': {'token_usage': {'completion_tokens': 51,
   'prompt_tokens': 21,
   'total_tokens': 72},
  'model_name': 'gpt-4o-mini-2024-07-18',
  'system_fingerprint': 'fp_0f03d4f0ee',
  'finish_reason': 'stop',
  'logprobs': None},
 'type': 'ai',
 'name': None,
 'id': 'run-6a931b68-1755-45d2-a5f5-4955d342c6a5-0',
 'example': False,
 'tool_calls': [],
 'invalid_tool_calls': [],
 'usage_metadata': {'input_tokens': 21,
  'output_tokens': 51,
  'total_tokens': 72}}

#### Tools

In the previous section, we mentioned that agents can choose a combination of tools at their disposal to solve a particular problem, with LLMs as reasoning engines under the hood. 

LangChain offers integrations with dozens of popular APIs and services as agent-compatible tools. Most of them are available under the `langchain_community` package while some are inside `langchain_core`. 

For example, here is how you can use the ArXiv tool to answer scientific questions based on papers submitted to arXiv:

In [18]:
!pip install -Uq langchain_community arxiv

In [24]:
from langchain_community.tools import ArxivQueryRun

tool = ArxivQueryRun()

tool.invoke('Photosythesis')

'Published: 2012-08-22\nTitle: Reconstruction of the wavefunctions of coupled nanoscopic emitters using a coherent optical technique\nAuthors: M. Richter, F. Schlosser, M. Schoth, S. Burger, F. Schmidt, A. Knorr, S. Mukamel\nSummary: We show that using coherent, spatially resolved spectroscopy, complex hybrid\nwave functions can be disentangled into the individual wave functions of the\nindividual emitters. This way, detailed information on the coupling of the\nindividual emitters, not available in far-field spectroscopy can be obtained.\nThe proposed quantum state tomography relies on the ability to selectively\nexcite each emitter individually by spatially localized pulses. Simulations of\ncoupled semiconductor Ga/InAs quantum dots, using light fields available in\ncurrent nanoplasmonics, show that even undesired resonances can be removed from\nmeasured spectra. The method can also be applied to study the internal coupling\nof pigments in photosythesis and artificial light harvesting

We will use a few other tools when building our own agents. 

### Prompt templates

The most efficient way to query chat models is by using prompt templates. They allow you to structure your queries consistently and dynamically insert variables, making your interactions with the model more flexible and reusable.

In LangChain, there are many types of prompt templates with the most basic one being `PromptTemplate` class. It can be used with language (plain text) models:

In [28]:
from langchain_core.prompts import PromptTemplate

query_template = "When was {car_brand} founded?"
prompt = PromptTemplate(input_variables=["car_brand"], template=query_template)

prompt.invoke({"car_brand": "Audi"})

StringPromptValue(text='When was Audi founded?')

We can use this prompt template by chaining it to our language model:

In [30]:
from langchain_openai import OpenAI

llm = OpenAI(api_key=api_key)

# Create a chain
chain = prompt | llm

# Invoke the chain
output = chain.invoke({'car_brand': 'Chevrolet'})
print(output)



Chevrolet was founded on November 3, 1911.


The pipe operator (`|`) is part of LangChain Expression Language (LCEL) designed to chain multiple LangChain components and tools (more on LCEL later). 

Another prompt template class works with chat models:

In [34]:
from langchain_core.prompts import ChatPromptTemplate

chat_model = ChatOpenAI(api_key=api_key, model='gpt-4o-mini')

## H2: Setting Up Your Environment
### H3: Prerequisites
- Required knowledge and tools for working with LangChain Agents.
- Necessary libraries and setup instructions.


## H2: Building a Basic LangChain Agent
### H3: Initializing tools and models
- Step-by-step guide to setting up tools like search engines and web fetchers.
### H3: Creating and configuring an agent
- Instructions on creating an agent, defining its actions, and configuring it with tools.
- Example code snippets and explanations.


## H2: Advanced Agent Configuration
### H3: Adding memory and context
- How to incorporate conversational memory to make agents more interactive.
- Examples of different memory options and their applications.
### H3: Using multiple tools and actions
- Configuring agents to utilize multiple tools for more complex tasks.
- Code examples demonstrating multi-tool interactions.


## H2: Optimizing and Debugging Agents
### H3: Performance tuning
- Tips and techniques for optimizing the performance of LangChain Agents.
- Common issues and troubleshooting strategies.
### H3: Debugging tools and methods
- Utilizing LangSmith and other tools for monitoring and debugging agents.


## H2: Applications and Use Cases
### H3: Industry-specific applications
- Examples of how LangChain Agents can be applied in different industries such as finance, healthcare, and marketing.
### H3: Case studies
- Detailed case studies showcasing successful implementations of LangChain Agents.


## H2: Future Trends and Developments
### H3: Emerging features
- Upcoming updates and new features in LangChain.
### H3: Long-term vision
- The future potential of LangChain Agents in AI and their role in advancing automation.


## H2: Conclusion


## Setup

```bash
$ conda create -n langchain python=3.9 -y
$ conda activate langchain
$ pip install langchain langchain_openai langchain_community ipykernel python-dotenv
$ ipython kernel install --user --name=langchain
```

```bash
$ touch .env 
$ vim .env  # Paste your OPENAI key
```

```python
from dotenv import load_dotenv()
```

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv('OPENAI_API_KEY')

In [2]:
from langchain_openai import OpenAI

llm = OpenAI(openai_api_key=api_key)

question = "Is Messi the best footballer of all time?"
output = llm.invoke(question)

print(output)



Many people consider Lionel Messi to be one of the greatest footballers of all time. He has won numerous individual and team awards, broken multiple records, and has consistently performed at a high level for both club and country. However, the title of "best footballer of all time" is subjective and there are many other players who have also been considered for this title, such as Diego Maradona, Pele, and Cristiano Ronaldo. Ultimately, it is a matter of personal opinion.


### Stuff I should cover
- Difference between an LLMs and chat models in langchain. [use this link](https://python.langchain.com/v0.1/docs/modules/model_io/)
- Prompt templates (see link above)
- Output parses (see link above)
- [Retrieval](https://python.langchain.com/v0.1/docs/modules/data_connection/)
- 

## Conclusion

In this article, we explored what makes LangChain agents distinct from chains and the important building blocks used in constructing them. We first introduced what agents are and how they differ from the more traditional chain constructs regarding flexibility and capability of making decisions. 

Then we looked at the key components you need to know about in order to build an agent: chat models, tools, and prompt templates. Finally, we ran through two examples demonstrating how to build simple and advanced agents. Natural language processing is developing continually, and LangChain agents are at the forefront of this progression, paving the way for an even more intelligent and versatile family of AI.

Here are some related resources to increase your LangChain:

- https://www.datacamp.com/courses/developing-llm-applications-with-langchain
- https://www.datacamp.com/tutorial/prompt-engineering-with-langchain
- https://www.datacamp.com/tutorial/how-to-build-llm-applications-with-langchain
- https://www.datacamp.com/tutorial/building-a-gpt-model-with-browsing-capabilities-using-lang-chain-tools

Thank you for reading!